# Initial naive modeling
This is an effort to try creating a simple vectorfield to see if the discontinuities are plausible source locations. Each sensor location in the bounding box becomes a node with four vectors attached, one at each of the cardinal points of the compass. If the wind is blowing to the North, the vector attached to the north side of the node will be shown as pointing away from the node (an emitter) if the pollution is elevated; otherwise the vector will be shown pointing toward the node (a sink). In this scenario, we would expect to see this where P is a source of pollution and N are nodes:  
<-- N <-- P --> N -->

In [1]:
# import libraries
import pandas as pd
import numpy as np 
import warnings
import sys
sys.path.append("../SingleDayAnalysis/")
from getData import get_data

warnings.filterwarnings('ignore')

In [2]:
# for now, use one day of data from parquet file
# grab sep27_full.parquet from the shared google drive

# Ben's local path to the parquet file
datafolder = "../my_stash/data"

In [3]:
# data_df = pd.read_parquet("{}/sep27_full.parquet".format(datafolder))

In [4]:
# this call to get_data function that will take bounding box and timeframe and return cleaned data

UP_LEFT = (38.008050, -122.536985)    # (lat, lon)
UP_RIGHT = (38.008050, -122.186437)   # (lat, lon)
DOWN_RIGHT = (37.701933, -122.186437) # (lat, lon)
DOWN_LEFT = (37.701933, -122.536985)  # (lat, lon)
START_DATE = '2019/09/27' # begin date to start taking data
END_DATE = '2019/09/27'   # end date to start taking data
START_HOUR = '10'         # hour EACH DAY to start, this allows us to control for time of day effects
END_HOUR = '18'           # hour EACH DAY to end, this allows us to control for time of day effects

data_df = get_data(UP_LEFT, UP_RIGHT, DOWN_RIGHT, DOWN_LEFT, START_DATE, END_DATE, START_HOUR, END_HOUR)

File 20190927.parquet does not exist


In [ ]:
# remove rows with na data for 2_5um
data_df = data_df[data_df['2_5um'].notna()]

In [ ]:
# calculate average counts by sensor
average_col = data_df.groupby(['sensor_id'])['2_5um'].mean() 
data_df = data_df.set_index(['sensor_id'])
data_df['avg_2_5um'] = average_col
data_df = data_df.reset_index() 

In [ ]:
# define direction degree range
NORTH = (316,45)
EAST = (46,135)
SOUTH = (136,225)
WEST = (226,315)

In [ ]:
data_df.head()

In [ ]:
# go through the dataframe and add new categorical column that indicates direction: 
# North, South, East, West, No wind, Missing, ERROR

wind_compass = [] 
for row in range(len(data_df)):
    try:
        degree = int(data_df.loc[row].wind_direction)
    except:
        wind_compass.append('Missing')
        continue
    if data_df.loc[row].wind_speed == 0:
        wind_compass.append('No wind')
    elif degree >= NORTH[0] and degree <= NORTH[1]:
        wind_compass.append('North')
    elif degree >= EAST[0] and degree <= EAST[1]:
        wind_compass.append('East')
    elif degree >= SOUTH[0] and degree <= SOUTH[1]:
        wind_compass.append('South')
    elif degree >= WEST[0] and degree <= WEST[1]:
        wind_compass.append('West')
    else:
        wind_compass.append('ERROR')
data_df['wind_compass'] = wind_compass

In [ ]:
# take a look at the counts. this is one day, so it may not show all possibilities.
data_df.groupby('wind_compass').count()

In [ ]:
# calculate average by compass counts

# similar to above but need to do additional groupby on new categorical column 'wind_from_compass'
# loop through each record, do a trig calculation based on angle to true direction, and sum up. 
# divide for avg and subtract the mean for the sensor from this average to get a +/- vector
# add the five different vector values in a new column called 'vector'

In [ ]:
# map the sensors on google maps api

In [ ]:
# map the four vectors emerging from each direction off the sensor point